### Module 5E - Disease Contamination

Written for AIFS.   
Copyright 2023 Tarek Zohdi, Emre Mengi, Omar Betancourt, Payton Goodrich. All rights reserved.

The pandemic of 2020 has led to a huge interest of modeling and simulation of infectious diseases. One of the central questions is the potential infection zone produced by a cough. In this assignment, you will develop mathematical models to simulate the progressive time-evolution of the distribution of particles in space produced by a cough. You will be asked that you recommend safe distancing guidelines based on the range, distribution and settling time of the particles under the influence of gravity and drag from the surrounding air.

<img src="cough.png" width="800" />

In [ ]:
################################## Importing Packages ####################################################

import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from copy import deepcopy
from matplotlib import animation
from matplotlib import rc
plt.rcParams.update({'font.size': 18})
from scipy.stats import uniform
import pandas as pd
from IPython.display import display, HTML
import Zohdi_Utils as zu

### **Problem 1:** Theory-Based Exercises ###

Answer the following questions *prior* to coding the assignment to better understand the background physics and mathematics that govern the given models. You **may** solve these problems by hand **and/or** using computational tools such as *Python* etc. Please include all handwritten work and code used to solve each problem.

**Problem 1.1:** Given the particles are spherical with a random distribution of radii $R_i, i = 1, 2, 3, ..., N = \textit{particles}$ and the masses are given by $m_i = \rho_i \frac{4}{3}\pi R^3_i$, where $\rho_i$ is the density of the particles and is assumed to be constant for all droplets, the equation of motion for the ith particle in the system is
    
\begin{equation}
    m_i \ddot{\mathbf{r}}_i = \mathbf{\Psi}_i^{grav} + \mathbf{\Psi}_i^{drag}
\end{equation}
    
where $m_i$ is the mass of each droplet, $\mathbf{\ddot{r}}_i$ is the droplet's acceleration, and the two $\mathbf{\Psi}$ are gravitational and drag forces respectively.    

Let the $z$ axis be the vertical axis so we note that the gravitation force be defined as 

\begin{equation}
    \mathbf{\Psi}_i^{grav} = m_i (g_x, g_y, g_z) = (0, 0, -m_ig)
\end{equation}

where $g$ is the gravitational constant. The drag force depends on the geometry of the droplet and the properties of the surrounding medium:

\begin{equation}
    \mathbf{\Psi}_i^{drag} = \frac{1}{2}\rho_a C_D ||\mathbf{v}^f - \mathbf{v}_i|| (\mathbf{v}^f - \mathbf{v}_i) A_i
\end{equation}

where $C_D$ is the drag coefficient, $A_i$ is the reference area, which for a sphere is $A_i = \pi R^2_i$, $\rho_a$ is the density of the ambient fluid environment and $\mathbf{v}^f$ is the velocity of the surrounding medium, which in this case is air.
\\*

Solve for the acceleration of the particle and plug in the individual equations for $\mathbf{\Psi}_i^{grav}$, $\mathbf{\Psi}_i^{drag}$, and $m$.


*Answer here*

**Problem 1.2:** To determine the drag coefficient, we first determine the Reynolds number of the droplet $Re$:

\begin{equation}
    Re = \frac{2 R \rho_a \Vert \bm{v}^f - \bm{v}_i \Vert}{\mu_f}
\end{equation}

Where $\mu_f$ is the viscosity of the surrounding medium. Then, the drag coefficient is a piecewise function of $Re$:

\begin{equation}
    C_{Di} = 
    \begin{cases}
    \frac{24}{Re}, & 0 < Re \leq 1 \\
    \frac{24}{Re^{0.646}}, & 1 < Re \leq 400 \\
    0.5, & 400 < Re \leq 3 \times 10^5 \\
    0.000366Re^{0.4275}, & 3 \times 10^5 < Re \leq 2 \times 10^6 \\
    0.18, & Re > 2 \times 10^6
    \end{cases}
\end{equation}

Given the variable glossary values, write the piece-wise drag coefficient in terms of particle velocity.

*Answer here*

**Problem 1.3:** We have the governing equation for the motion of the droplets, which we will numerically integrate using the Forward Euler method. Write down the Forward Euler scheme to integrate and find the position and velocity of some $i$-th droplet.

*Answer here*

### **Problem 2:** Coding Exercises ###  
Use the given python notebook template to complete the following coding exercises.

**Problem 2.1:** Define the constants used in the simulation. Use the variable glossary at the end of the assignment.

In [ ]:
# ################################## Problem 2.1 ####################################################

## System Parameters
Ptot = #FILL IN HERE # Total power per unit area (W/m^2) 
Nr = #FILL IN HERE # Number of light rays
sReg = #FILL IN HERE # domain size of solar rays (m)
hReg = #FILL IN HERE # initial height of solar rays (m)
Pmin = #FILL IN HERE #fraction of total power at which we stop tracking a ray

# cutoff ratios for ground absorption
Gm = #FILL IN HERE
Gp = #FILL IN HERE

domLim = #FILL IN HERE # Limit of domain in e1 and e2 domain (m)
c = #FILL IN HERE # Speed of light in a vacuum (m/s)
thetaR = #FILL IN HERE # Incoming angle of light (radians)

## Genetic Algorithm Parameters
K = #FILL IN HERE # Strings generated by breeding
P = #FILL IN HERE # Surviving strings for breeding
S = #FILL IN HERE # Design strings per generation
G = #FILL IN HERE # Total Generations
numLam = #FILL IN HERE # Number of parameters per design string
w1 = #FILL IN HERE # Cost function weight
w2 = #FILL IN HERE # Cost function weight

# Search Bounds
theta1SB = #FILL IN HERE # Solar Panel rotation about the e1-axis (radians)
theta2SB = #FILL IN HERE # Solar Panel rotation about the e2-axis (radians)
theta3SB = #FILL IN HERE # Solar Panel rotation about the e3-axis (radians)
ngSB = #FILL IN HERE # Refractive index of ground
nsSB = #FILL IN HERE # Refractive index of solar panel
R1SB = #FILL IN HERE # Solar Panel size parameter w.r.t. the e1-axis
R2SB = #FILL IN HERE # Solar Panel size parameter w.r.t. the e2-axis
R3SB = #FILL IN HERE # Solar Panel size parameter w.r.t. the e3-axis
p1SB = #FILL IN HERE # Solar Panel exponent w.r.t. the e1-axis
p2SB = #FILL IN HERE # Solar Panel exponent w.r.t. the e2-axis
p3SB = #FILL IN HERE # Solar Panel exponent w.r.t. the e3-axis
h0SB = #FILL IN HERE # Solar Panel height (m)

# max panel size parameters
R1max = R1SB[1]
R2max = R2SB[1]
R3max = R3SB[1]

# min panel exponent parameters
p1min = p1SB[0]
p2min = p2SB[0]
p3min = p3SB[0]

# ##################################################################################################################

**Problem 2.2:** Let us assume the total mass of all of the droplets in a cough is $M^{total} = 0.00005kg$, with a  mean particle radius of $\bar{R} = 0.0001 m$ with variations according to 

\begin{equation}
    R_i = \bar{R} \times (1 + A \times \xi_i)
\end{equation}

where $A = 0.9975$ and a random variable $-1 \leq \xi_i \leq 1$. The algorithm used for particle generation should be:

-Initialize total mass: $M = 0$

-Start loop: \quad$i=1,P_n$

-Compute a random deviation from the mean radius $\bar{R}$:  \quad $R_i = \bar{R} \times (1 + A \times \xi_i)$

-Compute the mass of the new particle and add to the total mass computed: \quad $M = M + m_i = M + \rho_i \frac{4}{3} \pi R^3_i$

-Check if the threshold for total mass has been reached: \quad If $M \geq M^{total}$ then stop (\textbf{determines $P_n =$ particles})

-End loop.

Initialize the particles size and mass randomly using the described algorithm.

**Problem 2.3:** The initial trajectories we determined from the following algorithm:


-Specify the relative direction of the cough: $\mathbf{N}^c = (N^c_x, N^c_y, N^c_z)$,

-For each particle, $i = 1, 2, 3, ..., P_n$, construct a (perturbed) trajectory vector:
    
\begin{equation}
    \mathbf{N}_i = (N^c_x + A^c_x \times \eta_{ix}, N^c_y + A^c_y \times \eta_{iy}, N^c_z + A^c_z \times \eta_{iz}), 
\end{equation}
    
where the random cone parameters are bounded: $-1 \leq \eta_{ix} \leq 1$, $0 \leq \eta_{iy} \leq 1$, $-1 \leq \eta_{iz} \leq 1$. 
    
-For each particle, normalize the trajectory vector:

\begin{equation}
    \mathbf{n}_i = \frac{1}{||\mathbf{N}_i||_2} (N_{ix}, N_{iy}, N_{iz}). 
\end{equation}
    
-For each particle, the velocity vector is constructed by a projection onto the normal vector:

\begin{equation}
    \mathbf{v}_i = V^c \mathbf{n}_i
\end{equation}

Initialize particle velocities using the described algorithm.

**Problem 2.4:** Fill in the lines marked "FILL IN HERE" in the main simulation cell and run the simulator.

### **Problem 3:** Analyzing Your Results ###  
Answer the following questions about the code you created.

**Problem 3.1:** Report approximately how far the particles landed from the starting point. Does the simulation agree with the prescribed '6-feet' social distancing recommendation from authorities?

*Answer here*

**Problem 3.2:** Knowing the particle size range used in the simulation, what percentage of the particles contaminating the air can be captured by a standard N95 mask? 

*Answer here*

**Problem 3.3:** Include a series of plots showing how the cough particles spread. Include 5 approximately evenly-spaced frames from $t = 0$ until the final time for your system.

In [ ]:
################################# Plotting Best Solution Animation ###############################################################

zu.PlotSolarAnimation(Lam[:,0], Nr, c, w1, w2, domLim, Ptot, Gm, Gp, Pmin, thetaR,R1max,R2max,R3max,p1min,p2min,\
            p3min,sReg,hReg)

Use the following variable glossary to fill in the required variables.

<img src="VariableGlossary.png" width="800" />